# Análise do Impacto das Emendas PIX na Porcentagem de Votos Válidos (2024)
Bruno Caetano Oliveira de Melo · MBA USP/ESALQ – Data Science & Analytics

Este notebook segue o roteiro acordado:
1. **Preparação dos dados**
2. **Análise exploratória**
3. **Modelos estimados** – OLS com efeitos fixos de partido _vs._ Modelo multinível
4. **Comparação de desempenho**
5. **Visualizações e interpretação**

**Dependências**: pandas, numpy, matplotlib, seaborn, statsmodels, tqdm, scipy

> Ajuste seu `PYTHONPATH` ou crie um ambiente virtual com `requirements.txt` do projeto.

---

In [ ]:
# Imports e configuração global
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col
from scipy.stats.mstats import winsorize
from scipy import stats

pd.options.display.float_format = "{:.3f}".format
sns.set_theme(style="whitegrid")

## 1 · Carregamento e preparação dos dados

In [ ]:
# Caminho da base previamente unificada
DATA_PATH = "../data/dados_com_clusters.csv"
base = pd.read_csv(DATA_PATH)

# Renomeia colunas para facilitar leitura
base = base.rename(columns={
    "emendas_pix_per_capita_partido_prefeito_eleito": "emendas_pix_pc",
    "porcentagem_votos_validos_2024": "perc_votos",
})

# Garante proporção em [0,1]
base["perc_votos"] = base["perc_votos"].clip(0, 1)

# Flag único candidato (100 % dos votos válidos)
base["unico_cand"] = base["perc_votos"].eq(1)

# Remove para análise principal
analise = base.loc[~base["unico_cand"].copy()]

# Flag zero de emendas
analise["sem_emenda"] = analise["emendas_pix_pc"].eq(0).astype(int)

# Winsorização (1 % em cada cauda) – evita distorções de outliers
analise["emendas_pix_pc_w"] = winsorize(analise["emendas_pix_pc"], limits=[0.01, 0.01])

# Transformação log(1+x) para valores >0
analise["log_emenda"] = np.log1p(analise["emendas_pix_pc_w"])

# Converte cluster bool → int
for c in ["cluster_0", "cluster_1", "cluster_2", "cluster_3"]:
    if c in analise.columns:
        analise[c] = analise[c].fillna(False).astype(int)

### 1.1 Resumo pós‑tratamento

In [ ]:
print(f"Observações totais: {len(base):,}")
print(f"Removidas por candidato único: {base['unico_cand'].sum():,}")
print(f"Amostra para análise: {len(analise):,}")
print(f"Zeros de Emenda PIX: {(analise['sem_emenda'].mean()*100):.1f}% da amostra")

## 2 · Análise exploratória

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 4))
# Distribuição original
sns.histplot(analise["emendas_pix_pc"], bins=40, kde=True, ax=ax[0])
ax[0].set_title("Distribuição Emendas PIX per capita (original)")
ax[0].set_xlabel("R$")
# Distribuição log(1+x) nos positivos
sns.histplot(analise.loc[analise["emendas_pix_pc"]>0, "log_emenda"], kde=True, ax=ax[1])
ax[1].set_title("Distribuição log(1+Emendas) nos >0")
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
sns.scatterplot(x="emendas_pix_pc", y="perc_votos", data=analise, alpha=0.6, ax=ax)
ax.set_xlabel("Emendas PIX per capita (R$)")
ax.set_ylabel("% Votos Válidos")
ax.yaxis.set_major_formatter(plt.matplotlib.ticker.PercentFormatter(1))
ax.set_title("Emendas PIX × Desempenho Eleitoral")
plt.tight_layout()

## 3 · Modelagem
Estimaremos:
* **OLS** com efeitos fixos de partido (`C(sigla_partido)`).
* **Multinível**: intercepto e inclinação aleatórios por partido.

In [ ]:
# Fórmula base – controles somente disponíveis na base
controles = "+ cluster_0 + cluster_1 + cluster_2 + cluster_3"

formula_fe = f"perc_votos ~ sem_emenda + log_emenda {controles} + C(sigla_partido_prefeito_eleito)"
fe_model = smf.ols(formula_fe, data=analise).fit(cov_type="HC3")  # robust
print(fe_model.summary())

In [ ]:
# Modelo multinível
formula_re = f"perc_votos ~ sem_emenda + log_emenda {controles}"
re_model = smf.mixedlm(
    formula_re,
    data=analise,
    groups=analise["sigla_partido_prefeito_eleito"],
    re_formula="1 + log_emenda",
).fit()
print(re_model.summary())

### 3.1 Teste LR para efeitos aleatórios

In [ ]:
ll_fe = smf.ols(formula_re + " + C(sigla_partido_prefeito_eleito)", data=analise).fit().llf
lr_stat = -2 * (ll_fe - re_model.llf)
p_val = stats.chi2.sf(lr_stat, df=2)  # 2 parâmetros aleatórios (interc + slope)
print(f"LR stat={lr_stat:.2f} – p-value={p_val:.4f}")

## 4 · Comparação de desempenho dos modelos

In [ ]:
comparison = summary_col([fe_model, re_model], stars=True, float_format="%.3f", model_names=["FE Partido", "Multinível"])
print(comparison)

### 4.1 Componente Logístico (Hurdle – parte 1)

In [ ]:
logit_formula = f"sem_emenda ~ {controles} + C(sigla_partido_prefeito_eleito)"
logit_model = smf.logit(logit_formula, data=analise).fit()
print(logit_model.summary())

## 5 · Visualizações de coeficientes

In [ ]:
coef_df = pd.DataFrame({
    "term": fe_model.params.index,
    "coef": fe_model.params.values,
    "se": fe_model.bse,
}).query("term not in ['Intercept'] and term.str.startswith('C(') == False", engine="python")

plt.figure(figsize=(6, 4))
plt.errorbar(coef_df["coef"], coef_df["term"], xerr=1.96*coef_df["se"], fmt="o")
plt.axvline(0, color="black", lw=1, ls="--")
plt.title("Coeficientes (OLS – efeitos fixos)")
plt.xlabel("Estimativa ± 1.96 SE")
plt.tight_layout()

## 6 · Diagnóstico de resíduos (OLS)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))
# Resíduos vs Ajustado
sns.scatterplot(x=fe_model.fittedvalues, y=fe_model.resid, alpha=0.5, ax=ax[0])
ax[0].axhline(0, ls="--", c="red")
ax[0].set_xlabel("Valores ajustados")
ax[0].set_ylabel("Resíduos")

# QQ‑Plot
sm.qqplot(fe_model.resid, line="45", fit=True, ax=ax[1])
ax[1].set_title("QQ‑Plot dos Resíduos")
plt.tight_layout()

## 7 · Conclusões
* As emendas PIX apresentam **(i) impacto significativo / não significativo** … (interpretar com base em p‑value e sinal).
* O teste LR indica que **(não) há** variância entre partidos além da capturada pelos efeitos fixos.
* Resíduos aproximadamente **(normais / heterocedásticos)**.
* Municípios sem emenda acumulam … (descrever efeito do `sem_emenda`).

> Documente estes achados na seção de resultados do TCC, mencionando limitações (zeros excessivos, ausência de algumas variáveis, etc.) e possíveis extensões (Beta regression, zero‑inflated models).

### Glossário de variáveis
| Sigla | Descrição |
|-------|-----------|
| perc_votos | Proporção de votos válidos (0‑1) |
| emendas_pix_pc | Total de Emendas PIX per capita (R$) |
| sem_emenda | 1 se `emendas_pix_pc`==0, 0 caso contrário |
| log_emenda | log(1+emendas_pix_pc_w) |
| cluster_* | Dummies de clusters socioeconômicos |
| sigla_partido_prefeito_eleito | Agrupamento/efeito de partido |

---
💡 **Próximos passos**
* Testar Beta regression (distribuição de proporções) e Zero‑Inflated Gamma para checar robustez.
* Avaliar autocorrelação espacial dos resíduos (Moran’s I).
* Explorar variáveis adicionais se disponíveis (IDHM, PIB pc, densidade, etc.).